In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.metrics import silhouette_samples
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans
from sklearn.decomposition import PCA
from itertools import product
from scipy.cluster.hierarchy import dendrogram, linkage

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
trees = pd.read_csv('/kaggle/input/tree-census-2015-in-nyc-cleaned/tree_census_processed.csv', index_col='tree_id')
print(trees.shape)
trees.head()

# 1. Data analysis

In [ ]:
trees[['tree_dbh', 'stump_diam']].describe()

In [ ]:
cat_features = []
two_features = []
for i in trees.columns:
    if i not in (['tree_dbh', 'stump_diam']):
        if trees[i].nunique()>2: cat_features.append(i)
        else: two_features.append(i)

In [ ]:
print(*cat_features, sep='; ')
print(*two_features, sep='; ')

In [ ]:
for i in cat_features:
    print('**************************************************')
    print(trees[i].value_counts())
    print('**************************************************')

In [ ]:
trees.problems.str.get_dummies(',')

In [ ]:
problems_feat = ['pr_BranchLights','pr_BranchOther','pr_MetalGrates', 'pr_None', 'pr_RootOther','pr_Sneakers', 'pr_Stones', 'pr_TrunkLights', 'pr_TrunkOther', 'pr_WiresRope']
trees[problems_feat] = trees.problems.str.get_dummies(',')
trees = trees.drop('problems', axis=1)

cat_features.remove('problems')
trees.head()

In [ ]:
oe = OrdinalEncoder()
trees[two_features] = oe.fit_transform(trees[two_features]).astype('int32')
trees.head()

In [ ]:
trees = pd.get_dummies(trees, columns = cat_features)
print(trees.shape)
trees.head()

In [ ]:
trees = StandardScaler().fit_transform(trees)

Сonvert the data to a two-dimensional format to increase the speed of the algorithms. The initial data set is too large, so only 50 thousand rows will be used in further work.

In [ ]:
X_emb = PCA(n_components=2, random_state=32).fit_transform(trees)[:50001]
del trees

# 2. KMeans 

In [ ]:
def kmeans_():
    cluster_numbers = range(1,9)
    inertia = []

    for i in cluster_numbers:
        kmeans = KMeans(n_clusters=i, random_state=32).fit(X_emb)
        _ = kmeans.predict(X_emb)
        inertia.append(kmeans.inertia_)

    sns.lineplot(x=cluster_numbers, y=inertia, markers=True)
    plt.title(f'Kmeans')
    plt.show()

In [ ]:
kmeans_()

In [ ]:
def silhouette_plot(est):
    predict_val = est.fit_predict(X_emb)
    clust_lab = np.unique(predict_val)
    n_cl = clust_lab.shape[0]
    silh_val = silhouette_samples(X_emb, predict_val, metric='euclidean')
    
    y_ax_lower, y_ax_upper = 0, 0
    yticks = []
    
    for c in clust_lab:
        c_silh_val = silh_val[predict_val == c]
        c_silh_val.sort()
        y_ax_upper += len(c_silh_val)
        color = ['orange', 'lightgreen', 'lightblue', 'yellow', 'green'][c]
        plt.barh(range(y_ax_lower, y_ax_upper), c_silh_val, height=1., edgecolor='none', color=color)
        yticks.append((y_ax_lower+y_ax_upper)/2.0)
        y_ax_lower += len(c_silh_val)
    silh_avg = np.mean(silh_val)
    plt.axvline(silh_avg, color='red', linestyle='--')
    plt.yticks(yticks, clust_lab+1)
    plt.ylabel('Cluster')
    plt.xlabel('Silhouette')
    plt.tight_layout()
    plt.show()
    
    return predict_val

In [ ]:
kmeans_model = KMeans(n_clusters=3, random_state=32)
predict_kmeans = silhouette_plot(kmeans_model)

# 3. Agglomerative clustering

In [ ]:
linked = linkage(np.array(X_emb), 'single')
plt.figure(figsize=(20,12))
dendrogram(linked, orientation='top',
           distance_sort='descending')
plt.show()

del linked

According to the constructed dendrogram, it is difficult to determine the number of clusters, so let's assume that there are 4 clusters.

In [ ]:
aggl_model = AgglomerativeClustering(n_clusters=4, linkage='single')
predict_aggl = silhouette_plot(aggl_model)

# 4. DBSCAN

In [ ]:
eps_values = np.arange(0.5, 2.5, 0.25)
min_samples = np.arange(2,11)
params = list(product(eps_values, min_samples))

clusters = []
sil_score = []

for p in params:
    dbscan_cl = DBSCAN(eps=p[0], min_samples=p[1]).fit(X_emb)
    clusters.append(len(np.unique(dbscan_cl.labels_)))
    sil_score.append(silhouette_score(X_emb, dbscan_cl.labels_))

In [ ]:
tmp = pd.DataFrame.from_records(params, columns=['eps','min_samples'])
tmp['sil_score'] = sil_score
pivot_t = pd.pivot_table(tmp, values='sil_score', index='min_samples',
                         columns='eps')
sns.heatmap(pivot_t,annot=True, yticklabels=min_samples,
            xticklabels=eps_values)
plt.show()

The global maximum is reached at *min_samples* equal to 2-3 and *eps* equal to 2.0-2.25.

In [ ]:
dbscan_model = DBSCAN(eps=3, min_samples=2.25)
predict_dbscan = silhouette_plot(dbscan_model)

# 5. Comparison of clustering results

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,5))

sns.scatterplot(ax=ax[0], x=X_emb[:,0], y=X_emb[:,1], hue=predict_kmeans)
ax[0].set_title('Kmeans')
sns.scatterplot(ax=ax[1], x=X_emb[:,0], y=X_emb[:,1], hue=predict_aggl)
ax[1].set_title('Agglomerative')
sns.scatterplot(ax=ax[2], x=X_emb[:,0], y=X_emb[:,1], hue=predict_dbscan)
ax[2].set_title('DBScan')

fig.tight_layout()
fig.show()